# Introduction to Programming
## Data wrangling - Lab 9
### Data preparation with string manipulation

Import the very messy data in `MMG-2018.xlsx` which stores the missing migrant data for 2018.
The data is stored in a single sheet per month. 
The data is pretty clean, but you still have a bit of preparation to do.
You want to unify the records, reconstuct the dates, create unique identifiers, replace the column coordinates with columns for latitude and longitude, and finally try to extract the country of an event from textual information.

Specifically, perform the following activities:

- Create a connection with the excel file.

- Write a for loop to iteratively import each sheet as a dataframe, remove redundant columns, and merge all dataframes together. Check that your code works as intended. For the purpose of the ensuing steps, it will be helpful to introduce a new column that stores the name of the month to which the data refer. This name can be found as the sheet name.

- Reconstruct the dates of each occurrence in the format `<4-digit year>/<2-digit month>/<2-digit day>, <name of day of the week>`. Sort the dataframe in ascending chronological order.


- Create unique identifiers for each record. These records should have the following structure `<year of the occurrence>.<4-digit unique integer label>`. For instance, the first record should be `2018.0001`.

- Split the column `Coordinates` into a two columns `Latitude` and `Longitude` (this is the order in which they appear in `Coordinates`). Check that these columns are numeric, and convert them otherwise. Remove the column `Coordinates`. Compute descriptive statistics for the occurrences in the boreal and austral hemispheres.

- Consider column `Location of death`. Note that the last word in each record often corresponds to the country of the event. Extract the last word and store it in a separate column `Country of event (tentative)`. In case you get here by the end of the lab: first, good job; second, try to refine the content of this column.

In [1]:
import pandas as pd

#### Unify dataframe

In [2]:
xls_file = pd.ExcelFile('MMG-2018.xlsx')

In [3]:
xls_file.sheet_names

['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec']

In [4]:
df = pd.DataFrame()

In [5]:
df

""


We loop over the names of the sheets, import the dataframes in each sheet using the `parse()` method of the `pd.ExcelFile` objects, and concatenate them vertically starting with an empty dataframe.

In [6]:
for sheet in xls_file.sheet_names:
    addendum = xls_file.parse(sheet)
#     alternatively:
#     addendum = pd.read_excel(xls_file, sheet)
    addendum['month'] = sheet
    df = pd.concat([df, addendum])

Check that this has worked properly.

In [7]:
df.head()

,Reported Day,Number Dead,Minimum Estimated Number of Missing,Total Number of Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Cause of Death,Location of death,Coordinates,month
0,1,4.0,NaN,4,2.0,1.0,3.0,NaN,Vehicle accident / death linked to hazardous t...,"BR-471 motorway near Santa Vit√≥ria do Palmar,...","-33.5176991, -53.3342095",Jan
1,2,1.0,NaN,1,NaN,NaN,1.0,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...,"32.264014, -111.661872",Jan
2,2,1.0,NaN,1,NaN,NaN,NaN,NaN,Mixed or unknown,"Ranch near Falfurrias, Brooks County, Texas, USA","27.2474839, -98.3788079",Jan
3,2,1.0,NaN,1,NaN,NaN,1.0,NaN,Mixed or unknown,"Mariposa Ranch near Falfurrias, Brooks County,...","27.1058052, -98.146774",Jan
4,2,NaN,1.0,1,1.0,NaN,NaN,NaN,Drowning,"Kolpa/Kupa river, near Ukovaci, municipality o...","45.5597062, 15.2988207",Jan


In [8]:
df.tail()

,Reported Day,Number Dead,Minimum Estimated Number of Missing,Total Number of Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Cause of Death,Location of death,Coordinates,month
94,28,1.0,NaN,1,NaN,NaN,1.0,NaN,Drowning,"Off the coast of Imperial Beach, San Diego Cou...","32.54188202, -117.1337341",Dec
95,30,14.0,NaN,14,12.0,NaN,NaN,NaN,Vehicle accident / death linked to hazardous t...,"Inside vehicle on Tanzania-Zambia highway, nea...","-6.840928719, 37.62619356",Dec
96,30,NaN,1.0,1,9.0,1.0,1.0,1.0,Drowning,"46 miles east of the Jupiter Inlet, Florida, USA","26.8304375, -79.5253125",Dec
97,31,1.0,NaN,1,NaN,NaN,NaN,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...,"32.27932, -112.8548",Dec
98,31,2.0,NaN,2,NaN,NaN,NaN,NaN,Mixed or unknown,"Yuma County, Arizona, USA","32.6926512, -114.6276916",Dec


In [9]:
df.isnull().all(axis=0).sum()

0

In [10]:
df.isnull().all(axis=1).sum()

0

In [11]:
df.dtypes

Reported Day                             int64
Number Dead                            float64
Minimum Estimated Number of Missing    float64
Total Number of Dead and Missing         int64
Number of Survivors                    float64
Number of Females                      float64
Number of Males                        float64
Number of Children                     float64
Cause of Death                          object
Location of death                       object
Coordinates                             object
month                                   object
dtype: object

#### Reconstruct dates

Use the `pandas` `str.cat()` method to concatenate strings. All elements in the concatenation need to be strings, so you need to cast any numbers to strings. This is similar to what you would do with the `+` operator between strings.

In [12]:
df['year'] = '2018'
df['Reported Day'].astype(str).str.cat([df['month'], df['year']], sep = '/')

0      1/Jan/2018
1      2/Jan/2018
2      2/Jan/2018
3      2/Jan/2018
4      2/Jan/2018
         ...     
94    28/Dec/2018
95    30/Dec/2018
96    30/Dec/2018
97    31/Dec/2018
98    31/Dec/2018
Name: Reported Day, Length: 1675, dtype: object

In [13]:
# alternatively
df[['Reported Day', 'month']].apply(lambda x : str(x[0]) + '/' + x[1] + '/2018', axis = 1) 

0      1/Jan/2018
1      2/Jan/2018
2      2/Jan/2018
3      2/Jan/2018
4      2/Jan/2018
         ...     
94    28/Dec/2018
95    30/Dec/2018
96    30/Dec/2018
97    31/Dec/2018
98    31/Dec/2018
Length: 1675, dtype: object

Cast to datetime.

In [14]:
df['date'] = pd.to_datetime(df['Reported Day'].astype(str).str.cat([df['month'], df['year']], sep = '/'))

In [15]:
df['date']

0    2018-01-01
1    2018-01-02
2    2018-01-02
3    2018-01-02
4    2018-01-02
        ...    
94   2018-12-28
95   2018-12-30
96   2018-12-30
97   2018-12-31
98   2018-12-31
Name: date, Length: 1675, dtype: datetime64[ns]

In [16]:
df.drop(['year', 'month', 'Reported Day'], axis = 1, inplace = True)

In [17]:
df.head()

,Number Dead,Minimum Estimated Number of Missing,Total Number of Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Cause of Death,Location of death,Coordinates,date
0,4.0,NaN,4,2.0,1.0,3.0,NaN,Vehicle accident / death linked to hazardous t...,"BR-471 motorway near Santa Vit√≥ria do Palmar,...","-33.5176991, -53.3342095",2018-01-01
1,1.0,NaN,1,NaN,NaN,1.0,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...,"32.264014, -111.661872",2018-01-02
2,1.0,NaN,1,NaN,NaN,NaN,NaN,Mixed or unknown,"Ranch near Falfurrias, Brooks County, Texas, USA","27.2474839, -98.3788079",2018-01-02
3,1.0,NaN,1,NaN,NaN,1.0,NaN,Mixed or unknown,"Mariposa Ranch near Falfurrias, Brooks County,...","27.1058052, -98.146774",2018-01-02
4,NaN,1.0,1,1.0,NaN,NaN,NaN,Drowning,"Kolpa/Kupa river, near Ukovaci, municipality o...","45.5597062, 15.2988207",2018-01-02


Use the function `datetime.strftime()` of the `datetime` builtin package to cast the dates into the desired format.

In [18]:
import datetime

In [19]:
df['date'].apply(lambda x : datetime.datetime.strftime(x, '%Y-%m-%d, %A'))

0      2018-01-01, Monday
1     2018-01-02, Tuesday
2     2018-01-02, Tuesday
3     2018-01-02, Tuesday
4     2018-01-02, Tuesday
             ...         
94     2018-12-28, Friday
95     2018-12-30, Sunday
96     2018-12-30, Sunday
97     2018-12-31, Monday
98     2018-12-31, Monday
Name: date, Length: 1675, dtype: object

In [20]:
def date_to_my_format(x) :
    return datetime.datetime.strftime(x, '%Y-%m-%d, %A')

In [21]:
df.iloc[0]['date']

Timestamp('2018-01-01 00:00:00')

In [22]:
date_to_my_format(df.iloc[0]['date'])

'2018-01-01, Monday'

In [23]:
df['date'].apply(date_to_my_format)

0      2018-01-01, Monday
1     2018-01-02, Tuesday
2     2018-01-02, Tuesday
3     2018-01-02, Tuesday
4     2018-01-02, Tuesday
             ...         
94     2018-12-28, Friday
95     2018-12-30, Sunday
96     2018-12-30, Sunday
97     2018-12-31, Monday
98     2018-12-31, Monday
Name: date, Length: 1675, dtype: object

In [24]:
df['date'] = df['date'].apply(lambda x : datetime.datetime.strftime(x, '%Y-%m-%d, %A'))

In [25]:
df.sort_values(by = 'date', inplace = True)
df.head(20)

,Number Dead,Minimum Estimated Number of Missing,Total Number of Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Cause of Death,Location of death,Coordinates,date
0,4.0,NaN,4,2.0,1.0,3.0,NaN,Vehicle accident / death linked to hazardous t...,"BR-471 motorway near Santa Vit√≥ria do Palmar,...","-33.5176991, -53.3342095","2018-01-01, Monday"
1,1.0,NaN,1,NaN,NaN,1.0,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...,"32.264014, -111.661872","2018-01-02, Tuesday"
2,1.0,NaN,1,NaN,NaN,NaN,NaN,Mixed or unknown,"Ranch near Falfurrias, Brooks County, Texas, USA","27.2474839, -98.3788079","2018-01-02, Tuesday"
3,1.0,NaN,1,NaN,NaN,1.0,NaN,Mixed or unknown,"Mariposa Ranch near Falfurrias, Brooks County,...","27.1058052, -98.146774","2018-01-02, Tuesday"
4,NaN,1.0,1,1.0,NaN,NaN,NaN,Drowning,"Kolpa/Kupa river, near Ukovaci, municipality o...","45.5597062, 15.2988207","2018-01-02, Tuesday"
5,2.0,NaN,2,NaN,NaN,2.0,NaN,Mixed or unknown,"Sahara desert, Libya","21.284256, 24.028913","2018-01-02, Tuesday"
6,1.0,NaN,1,NaN,NaN,1.0,NaN,Drowning,"Rio Bravo, at Donna Rio Bravo International Br...","26.0635489, -98.0760348","2018-01-03, Wednesday"
7,2.0,NaN,2,NaN,1.0,1.0,NaN,Mixed or unknown,"Sahara desert, Libya","24.642024, 14.587863","2018-01-03, Wednesday"
8,1.0,NaN,1,NaN,NaN,1.0,NaN,Mixed or unknown,"East Ranch-Santa Fe Feeders, near Encino, 20 m...","26.8077686, -98.118294","2018-01-04, Thursday"
9,1.0,NaN,1,NaN,NaN,1.0,NaN,Drowning,"Off the coast of Sanl√∫car de Barrameda, C√°di...","36.7846768, -6.5809352","2018-01-04, Thursday"


#### Unique identifiers

An easy solution is to use the `format()` function and formatting conventions, which however we haven't covered.

In [26]:
?format

Signature: format(value, format_spec='', /)
Docstring:
Return value.__format__(format_spec)

format_spec defaults to the empty string.
See the Format Specification Mini-Language section of help('FORMATTING') for
details.
Type:      builtin_function_or_method


In [27]:
id = [ '2018.' + format(x, '04d') for x in range(0, len(df))]

In [28]:
id[:12]

['2018.0000',
 '2018.0001',
 '2018.0002',
 '2018.0003',
 '2018.0004',
 '2018.0005',
 '2018.0006',
 '2018.0007',
 '2018.0008',
 '2018.0009',
 '2018.0010',
 '2018.0011']

You can easily work around the need for this function by defining a function of your own. 
This function creates a string of as many zeros as required to get to 4 digits, given a number.

In [29]:
def four_digit_code (n):
    n_digit = len(str(n))
    return '0' * (4 -  n_digit) + str(n)

In [30]:
id = [ '2018.' + four_digit_code(x) for x in range(0, len(df))]
id[:12]

['2018.0000',
 '2018.0001',
 '2018.0002',
 '2018.0003',
 '2018.0004',
 '2018.0005',
 '2018.0006',
 '2018.0007',
 '2018.0008',
 '2018.0009',
 '2018.0010',
 '2018.0011']

Here I am choosing to use the id as a new index.

In [31]:
df.index = id
df

,Number Dead,Minimum Estimated Number of Missing,Total Number of Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Cause of Death,Location of death,Coordinates,date
2018.0000,4.0,NaN,4,2.0,1.0,3.0,NaN,Vehicle accident / death linked to hazardous t...,"BR-471 motorway near Santa Vit√≥ria do Palmar,...","-33.5176991, -53.3342095","2018-01-01, Monday"
2018.0001,1.0,NaN,1,NaN,NaN,1.0,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...,"32.264014, -111.661872","2018-01-02, Tuesday"
2018.0002,1.0,NaN,1,NaN,NaN,NaN,NaN,Mixed or unknown,"Ranch near Falfurrias, Brooks County, Texas, USA","27.2474839, -98.3788079","2018-01-02, Tuesday"
2018.0003,1.0,NaN,1,NaN,NaN,1.0,NaN,Mixed or unknown,"Mariposa Ranch near Falfurrias, Brooks County,...","27.1058052, -98.146774","2018-01-02, Tuesday"
2018.0004,NaN,1.0,1,1.0,NaN,NaN,NaN,Drowning,"Kolpa/Kupa river, near Ukovaci, municipality o...","45.5597062, 15.2988207","2018-01-02, Tuesday"
...,...,...,...,...,...,...,...,...,...,...,...
2018.1670,1.0,NaN,1,NaN,NaN,1.0,NaN,Drowning,"Off the coast of Imperial Beach, San Diego Cou...","32.54188202, -117.1337341","2018-12-28, Friday"
2018.1671,14.0,NaN,14,12.0,NaN,NaN,NaN,Vehicle accident / death linked to hazardous t...,"Inside vehicle on Tanzania-Zambia highway, nea...","-6.840928719, 37.62619356","2018-12-30, Sunday"
2018.1672,NaN,1.0,1,9.0,1.0,1.0,1.0,Drowning,"46 miles east of the Jupiter Inlet, Florida, USA","26.8304375, -79.5253125","2018-12-30, Sunday"
2018.1673,1.0,NaN,1,NaN,NaN,NaN,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...,"32.27932, -112.8548","2018-12-31, Monday"


#### Lat and lon

In [32]:
df['Coordinates'].str.split(', ')

2018.0000     [-33.5176991, -53.3342095]
2018.0001       [32.264014, -111.661872]
2018.0002      [27.2474839, -98.3788079]
2018.0003       [27.1058052, -98.146774]
2018.0004       [45.5597062, 15.2988207]
                        ...             
2018.1670    [32.54188202, -117.1337341]
2018.1671    [-6.840928719, 37.62619356]
2018.1672      [26.8304375, -79.5253125]
2018.1673          [32.27932, -112.8548]
2018.1674     [32.6926512, -114.6276916]
Name: Coordinates, Length: 1675, dtype: object

A safer formulation would be to use the RegEx for any number of whitespaces `\s+`. 
Here however the formulation above is sufficient.

In [33]:
df['Coordinates'].str.split(',\s+')

2018.0000     [-33.5176991, -53.3342095]
2018.0001       [32.264014, -111.661872]
2018.0002      [27.2474839, -98.3788079]
2018.0003       [27.1058052, -98.146774]
2018.0004       [45.5597062, 15.2988207]
                        ...             
2018.1670    [32.54188202, -117.1337341]
2018.1671    [-6.840928719, 37.62619356]
2018.1672      [26.8304375, -79.5253125]
2018.1673          [32.27932, -112.8548]
2018.1674     [32.6926512, -114.6276916]
Name: Coordinates, Length: 1675, dtype: object

In [34]:
df['Latitude'] = df['Coordinates'].str.split(', ').str[0].astype(float)

In [35]:
df['Longitude'] = df['Coordinates'].str.split(', ').str[1].astype(float)

In [36]:
df['Boreal'] = df['Latitude'] > 0

In [37]:
df.groupby('Boreal').describe()

Number Dead                                                \
             count      mean       std  min  25%  50%  75%   max   
Boreal                                                             
False         53.0  2.188679  2.594792  1.0  1.0  1.0  2.0  14.0   
True        1555.0  1.967846  3.602015  1.0  1.0  1.0  2.0  64.0   

       Minimum Estimated Number of Missing             ...   Latitude  \
                                     count       mean  ...        75%   
Boreal                                                 ...              
False                                  3.0   5.333333  ...  -4.872675   
True                                 116.0  15.474138  ...  32.682062   

                  Longitude                                               \
              max     count       mean        std         min        25%   
Boreal                                                                     
False   -0.154338      52.0  29.832640  25.738864  -79.884198  32.715315   
True    51.193208    1622.0 -17.143743  57.526905 -117.133734 -97.517864   

                                          
              50%        75%         max  
Boreal                                    
False   34.128729  39.286525   45.100326  
True     7.965108  23.181283  120.955254  

[2 rows x 72 columns]

#### Tentative geography

In [38]:
location_split = df['Location of death'].str.split('\s+')
location_split

2018.0000    [BR-471, motorway, near, Santa, Vit√≥ria, do, ...
2018.0001    [Pima, Country, Office, of, the, Medical, Exam...
2018.0002    [Ranch, near, Falfurrias,, Brooks, County,, Te...
2018.0003    [Mariposa, Ranch, near, Falfurrias,, Brooks, C...
2018.0004    [Kolpa/Kupa, river,, near, Ukovaci,, municipal...
                                   ...                        
2018.1670    [Off, the, coast, of, Imperial, Beach,, San, D...
2018.1671    [Inside, vehicle, on, Tanzania-Zambia, highway...
2018.1672    [46, miles, east, of, the, Jupiter, Inlet,, Fl...
2018.1673    [Pima, Country, Office, of, the, Medical, Exam...
2018.1674                       [Yuma, County,, Arizona,, USA]
Name: Location of death, Length: 1675, dtype: object

In [39]:
df['Country of event (tentative)'] = location_split.apply(lambda x : x[len(x) - 1])
df['Country of event (tentative)']

2018.0000       Brazil
2018.0001    location)
2018.0002          USA
2018.0003          USA
2018.0004     Slovenia
               ...    
2018.1670          USA
2018.1671     Tanzania
2018.1672          USA
2018.1673    location)
2018.1674          USA
Name: Country of event (tentative), Length: 1675, dtype: object

In [40]:
df.head()

,Number Dead,Minimum Estimated Number of Missing,Total Number of Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Cause of Death,Location of death,Coordinates,date,Latitude,Longitude,Boreal,Country of event (tentative)
2018.0000,4.0,NaN,4,2.0,1.0,3.0,NaN,Vehicle accident / death linked to hazardous t...,"BR-471 motorway near Santa Vit√≥ria do Palmar,...","-33.5176991, -53.3342095","2018-01-01, Monday",-33.517699,-53.334210,False,Brazil
2018.0001,1.0,NaN,1,NaN,NaN,1.0,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...,"32.264014, -111.661872","2018-01-02, Tuesday",32.264014,-111.661872,True,location)
2018.0002,1.0,NaN,1,NaN,NaN,NaN,NaN,Mixed or unknown,"Ranch near Falfurrias, Brooks County, Texas, USA","27.2474839, -98.3788079","2018-01-02, Tuesday",27.247484,-98.378808,True,USA
2018.0003,1.0,NaN,1,NaN,NaN,1.0,NaN,Mixed or unknown,"Mariposa Ranch near Falfurrias, Brooks County,...","27.1058052, -98.146774","2018-01-02, Tuesday",27.105805,-98.146774,True,USA
2018.0004,NaN,1.0,1,1.0,NaN,NaN,NaN,Drowning,"Kolpa/Kupa river, near Ukovaci, municipality o...","45.5597062, 15.2988207","2018-01-02, Tuesday",45.559706,15.298821,True,Slovenia
